In [22]:
import socket
#import send
#import Path
import cv2 as cv
import numpy as np
import math
from numpy.lib.function_base import disp

# ip = "192.168.43.9"
# ip = '192.168.43.9'
# '''This ip is your server machine's ip '''
# port = 12362

def takepic():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video2 2


def send_common(text):

    c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    c.connect((ip, port))

    c.send(bytes(text, encoding="utf-8"))
    rep = c.recv(1000)
    print(str(rep,encoding="utf-8"))
    c.close()
    return str(rep, encoding="utf-8")


# def get_data_frame(text):
#     c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     c.connect((ip, port))
#     c.send(bytes(text, encoding="utf-8"))
#     rep = c.recv(1000)
#     c.close()

#     message = eval(rep)
#     Location = message['Location']
#     Barrier = message['Barrier']
#     Target = message['Target']
#     Direction = message['Direction']
#     Start = message['Start']
#     End = message['End']
#     T_direction = []
#     print(Target[0])
#     a = Target[0]
#     b = Target[1]
#     T_direction.append(Direction[a-1]) 
#     T_direction.append(Direction[b-1]) 
#     #T_direction[1] = Direction[Target[1]]
#     print('Location:', Location,'Barrier:', Barrier,'Target:', Target,'Direction',Direction,'Start:', Start, 'End:',End)
#     return Location, Barrier, Target, T_direction,Start, End



In [23]:
import binascii
import serial
import os
os.system('sh ./stop_sys_ttyPS0.sh')
def run_action(cmd):
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    cnt_err = 0
    while 1:
        test_read = ser.read()
        print('test_read', test_read)
        cnt_err += 1
        if test_read== b'\xa3' or cnt_err == 50:
            break
    
    if cnt_err == 50:
        print('can not get REQ')
    else:
        print('read REQ finished!')
        ser.write(cmd2data(cmd))
        print('send action ok!')
    ser.close()
    
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

def cmd2data(cmd):
    cnt=0
    cmd_list=[]
    for i in cmd:
        cnt+=1
        cmd_list+=[ord(i)]
    cmd_list=[0xff,0xff]+[(cnt+5)>>8,(cnt+5)&255]+[0x01,(cnt+1)&255,0x03]+cmd_list
    cmd_list=cmd_list+crc_calculate(cmd_list)
    return cmd_list

def wait_req():
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    while 1:
        test_read=ser.read()
        if test_read== b'\xa3' :
            print('read REQ finished!') 
            break


In [24]:
def obj_recog():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2
    # jupyter调用摄像头的代码
    im = cv.imread(orig_img_path)
    #cv.imshow('t',im)
    #cv.imgshow(im)
    width = im.shape[1]
    height = im.shape[0]
    rate = round(width*0.5)
    rate1 = round(height*0.2)
    half = round(height/2)
    ims = im[rate1:half,0:(width-rate)]
    ims_right = im[(half+1):(height-rate1),0:(width-rate)]
    lower_red = np.array([0, 43, 20]) # 红色范围低阈值
    upper_red = np.array([20, 255, 255]) # 红色范围高阈值
    lower_red2 = np.array([150, 43, 20]) # 红色范围低阈值
    upper_red2 = np.array([200, 255, 255]) # 红色范围高阈值
    hsv_img = cv.cvtColor(ims, cv.COLOR_BGR2HSV)
    mask_red = cv.inRange(hsv_img, lower_red, upper_red)
    mask_red2 = cv.inRange(hsv_img, lower_red2, upper_red2)
    mask = cv.bitwise_or(mask_red,mask_red2)
    # again for right side
    hsv_img_right = cv.cvtColor(ims_right, cv.COLOR_BGR2HSV)
    mask_red_right = cv.inRange(hsv_img_right, lower_red, upper_red)
    mask_red_right2 = cv.inRange(hsv_img_right, lower_red2, upper_red2)
    mask_right = cv.bitwise_or(mask_red_right,mask_red_right2)
    
    #print(mask_red)
    #print(mask_red2)
    #print(mask)
    #print(np.sum(mask_red))
    #print(np.sum(mask_red2))
    #print(np.sum(mask))
    nl = np.sum(mask)
    nr = np.sum(mask_right)
    #print(number)
    number = nl + nr
    
    h1 = height - rate1*2
    w1 = width - rate
    S = h1*w1
    print("nr",nr/255)
    print("nl",nl/255)
    test = nr/255 - nl/255
    print(test )
    print(number/S/255)
    number = (number/S/255)
    
    yu = 2000
    
    if(number < 0.01):
        kind =  0
    else:
        kind = 1
    #if (number>0.01 && number<0.03 && number>0.5):
        #kind = 

    return kind

In [35]:
def houfline(img):
    img = img[:,50:370]
    # cv.imshow("img",img)
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    l_blue = np.array([[100,43,46]])
    h_blue = np.array([[124,255,255]])
    l_green = np.array([[40,100,100]])
    h_green = np.array([[60,200,255]])
    mask_b = cv.inRange(hsv, l_blue, h_blue)
    mask_g = cv.inRange(hsv, l_green, h_green)
    res = cv.bitwise_or(mask_b, mask_g)
    # cv.imshow("test", res)
    # cv.imshow("mask", mask_g)
    list1 = []
    edges = cv.Canny(res,50,150,apertureSize = 3)
    # cv.imshow("mask2", edges)
    #lines = cv.HoughLinesP(mask_g,1,np.pi/180,100,minLineLength=100,maxLineGap=1)
    lines = cv.HoughLinesP(edges,1,np.pi/180,70,minLineLength=10,maxLineGap=100)
    if lines is None:
        return [0,0,0,0,0,0]
    for line in lines:
        x1,y1,x2,y2 = line[0]
        #print(x1,y1,x2,y2)
        m = (y1-y2)/(x1-x2)
        p = [0,240]
        a = y1-y2
        b = x2-x1
        c = x1*y2 - y1*x2
        dis = abs(a*p[0]+b*p[1]+c)/math.sqrt(a*a+b*b)
        #print(dis,m)
        
        flag = 0
        if (len(list1) == 0) :
            if(math.isinf(m)):
                break
            list1.append([m,dis])
        else:
            for i in range(0,len(list1)):
                if ( abs(math.atan(m)-math.atan(list1[i][0])) < 0.5 and abs(dis-list1[i][1])<200):
                    # list1[i][0] = (m+list1[i][0])/2
                    # list1[i][1] = (list1[i][1]+dis)/2
                    flag = 1
                    break
                elif(math.isinf(m)):
                    flag = 1
                    break
                    
            if (flag == 0):               
                list1.append([m,dis])
                     
            
        # cv.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    #cv.imwrite('houghlines3.jpg',img)
    
    # cv.imshow("img",img)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    #shape (640,480 3)
    print(list1)
    print(len(list1))
    nn = len(list1)
    mk = 0
    mr = 0
    lk = 0
    lr = 0
    rk = 0
    rr = 0
    kmax = 1.2
    kmin = -1
    mlmax = 300 #######TODO
    mr_big = 999
    for i in range(0,nn):
        if(abs(list1[i][0])>kmax and list1[i][1] < mr_big):
            mk = list1[i][0]
            mr = list1[i][1]
            mr_big = mr
        if(list1[i][0]<kmax and list1[i][0]>0):
            lk = list1[i][0]
            lr = list1[i][1]
        if(list1[i][0] > kmin and list1[i][0]<0):
            rk = list1[i][0]
            rr = list1[i][1]  
    print("all",[lk,lr,mk,mr,rk,rr])        
    return [lk,lr,mk,mr,rk,rr]





In [26]:
def center():
    ff = 0
    lrmin= 180 #######TODO
    rrmin=180  #######TODO
    mk_min = 10
    rmax = 100   #######TODO
    mrmax = 250
    mrmin = 110
    jishu = 0
    while(ff==0):
        takepic()
        img = cv.imread('/home/xilinx/jupyter_notebooks/common/data/webcam.jpg')
        [lk,lr,mk,mr,rk,rr] = houfline(img)
        if(mk==0 and jishu == 0):
            run_action('Back2Run')
            wait_req()
            print('back because mk==0')
            run_action('Stand')
            wait_req()
            jishu = 1
        elif(mk==0 and jishu == 1):
            run_action('Forwalk02')
            wait_req()
            jishu = 0
        elif(abs(mk)<mk_min):
            if(mk>0):
                #left
                run_action('turn001L')
                wait_req()
            elif(mk<0):    
                run_action('turn001R')
                wait_req()
        elif(mr<mrmin):
            run_action('Back2Run')
            #run_action('fastForward03')
            wait_req()
            print('back because too close')
            run_action('Stand')
            wait_req()
        elif(mr>mrmax):
            run_action('Forwalk02')########TODO
            #run_action('fastForward03')
            wait_req()
        elif(lk!=0 and rk!=0):
            if(abs(lr-rr)>rmax):
                if(lr>rr):
                    run_action('Left3move')
                    wait_req()
                elif(lr<rr):
                    run_action('Right3move')
                    wait_req()
            else:
                ff = 1        
        elif(lk==0 or rk == 0):
            if(lk!=0):
                if(lr<lrmin):
                    run_action('Right3move')
                    wait_req()    
            elif(rr!=0):
                if(rr<rrmin):
                     run_action('Left3move')
                     wait_req()
        else:
            run_action('Forwalk02')

In [6]:
center()

--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.


KeyboardInterrupt: 

In [27]:
def getpos(): # get an integer
    # get quarter aa bb cc
    # assume
    #(a1 a2)b1 b2,c1,c2
    #input : n1,n2
    m = (b2-a2)/(b1-a1)
    theta = math.atan(m)
    n1 = n1-a1
    n2 = n2-a2
    #define matrix
    #m1 m2
    #m3 m4
    pi = math.pi
    m1 = math.cos(pi - theta)
    m2 = -math.sin(pi - theta)
    #m3 = m1
    #m4 = -m2
    k1 = n1*m1 - n2*m1
    k2 = n1*m2 + n2*m1
    k1 = math.floor(k1/3)
    k2 = math.floor(k2/9)
    out = 3*k2+ k1
    return out

In [28]:
def get_location():   # need to be changed
    '''
    获取当前位置

    Returns:
        list: 位置坐标
    '''
    cmd = "robot1"  # 假定机器人是robot1
    Location_string = send_common(cmd)
    Location_num = int(Location_string)
    
    location = [Location_num//10%10, Location_num//1%10]
    return location


def walk_step1(current_point, next_point):
    '''
    向前走1格

    Args:
        current_point (list): 当前坐标点
        next_point (list): 下一个坐标点

    Returns:
        int: 1表示成功，0表示失败
    '''
    
    location_now = current_point
    L = 1
    while True:
        if location_now == current_point:
            cctl.run_action('SlowForward')
            location_now = get_location()
        else:
            for i in range(0, L):
                cctl.run_action('SlowForward')
            break
    return get_location()


def walk_step(current_point, next_point):
    '''
    向前走1格

    Args:
        current_point (list): 当前坐标点
        next_point (list): 下一个坐标点

    Returns:
        int: 1表示成功，0表示失败
    '''
    run_action('fastForward03')  
    wait_req()
    center()
    return next_point



def turn1(angle):
    '''
    转向

    Args:
        angle (int): 正表示顺时针，负表示逆时针，数值*90表示转动角度。如2表示顺时针转180度，-1表示逆时针转90度

    Returns:
        int: 1表示成功，0表示失败
    '''
    if angle != 0:
        cctl.run_action('SlowForward')
        time = [3, 3]
        if angle < 0:
            for i in range(0, -angle*time[0]):
                cctl.run_action('LeftTurn')
        else:
            for i in range(0, angle*time[1]):
                cctl.run_action('RightTurn')
    return 1


def turn(angle):
    if angle != 0:
        #run_action('SlowForward')
        time = [2, 2]
        if angle < 0:
            for i in range(0, -angle*time[0]):
                run_action('turn005L')
                wait_req()
                #center()
        else:
            for i in range(0, angle*time[1]):
                run_action('turn005R')
                wait_req()
        center()
    return 1





def read_map_info():
    data_frame = {"Location": [1, 4],"Barrier": [1, 5],"Target": [2, 5], "Start": [1, 2], "End": [3, 9]}
    return data_frame['Location'], data_frame['Barrier'], data_frame['Target'], data_frame['Start'], data_frame['End']





In [29]:
def heuristic_distace(Neighbour_node, target_node):
    x_dist = abs(Neighbour_node[0] - target_node[0])
    y_dist = abs(Neighbour_node[1] - target_node[1])
    H = x_dist + y_dist
    return H


def obs_position(number):
    obs_pos = []
    obstacle_dict = {
        1: [[1, 5]],
        2: [[5, 9]],
        3: [[9, 5]],
        4: [[5, 1]],
        5: [[3, 5], [4, 5]],
        6: [[5, 6], [5, 7]],
        7: [[6, 5], [7, 5]],
        8: [[5, 3], [5, 4]],
    }
    for n in number:
        obs_pos = obs_pos + obstacle_dict[n]
    return obs_pos

In [30]:
def djkstra(graph, start, end):
    path_set = set()    # 已求的路径集合
    priority_dic = {}
    for k in graph.keys():
        priority_dic[k] = [9999, False, ""] # 权重表构建为一个3维数组，分别是：最小路径代价，是否计算过最小边，最小路径
    priority_dic[start][0] = 0

    # 判断权重表中所有路点是否添加完毕
    def isSelectAll():
        ret = True
        for val in priority_dic.values():
            if not val[1]:
                ret = False
                break
        return ret

    while not isSelectAll():
        find_point = start
        find_path = start
        min_distance = 9999
        for path in path_set:
            end_point = path[-2:]
            path_distance = priority_dic[end_point][0]
            if path_distance < min_distance and not priority_dic[end_point][1]:
                find_point = end_point
                find_path = path
                min_distance = path_distance
        find_distance = priority_dic[find_point][0]
        neighbors = graph[find_point]
        for k in neighbors.keys():
            p = find_path + "-" + k
            weight = find_distance + neighbors[k]
            path_set.add(p)
            if weight < priority_dic[k][0]:
                priority_dic[k][0] = weight
                priority_dic[k][2] = p
        priority_dic[find_point][1] = True
        #print("prio",priority_dic)

    return priority_dic[end]


def path_finding(start_point, end_point, barrier):
    # start_point = get_location()
    # end_point = [2, 5]
    if start_point == end_point:
        return [start_point]
    obstacle_points = [[2, 2], [2, 5], [2, 8], [5, 2], [5, 5], [5, 8], [8, 2],
                       [8, 5], [8, 8]]
    obstacle_points.extend(obs_position(barrier))
    current_point = start_point
    path_vertices = [start_point]
    Neighbour_vertices = []

    while current_point != end_point:
        x = current_point[0]
        y = current_point[1]
        F = []  # 节点权值 F = g + h
        Neighbour_vertices = [
            [x - 1, y],
            [x, y - 1],
            [x, y],
            [x, y + 1],
            [x + 1, y],
        ]
        # 遍历相邻坐标
        for value in Neighbour_vertices:
            if value[0] in range(1, 10):
                if value[1] in range(1, 10):
                    if value not in obstacle_points + path_vertices:
                        # 如果满足节点 1, 在地图边界内 2, 不在障碍物点和已经经过的点, 计算权重
                        F.append(heuristic_distace(value, end_point) + 1)
                    else:
                        F.append(10000)
                else:
                    F.append(10000)
            else:
                F.append(10000)
        current_point = Neighbour_vertices[F.index(
            min(total_distance for total_distance in F))]
        path_vertices.append(current_point)
    return path_vertices


def path_finding_d(start_point, end_point, barrier):
    if start_point == end_point:
        return [start_point]
    obstacle_points = [[2, 2], [2, 5], [2, 8], [5, 2], [5, 5], [5, 8], [8, 2],
                       [8, 5], [8, 8]]
    obstacle_points.extend(obs_position(barrier))
    ################################################################
    graph = {}
    for i in range(1, 10):
        for j in range(1, 10):
            if [i, j] in obstacle_points:
                continue
            neighbor_weigh = {}
            neighbors = [[i - 1, j], [i, j + 1], [i + 1, j], [i, j - 1]]
            for neighbor in neighbors:
                if neighbor[0] < 1 or neighbor[0] > 9 or neighbor[1] < 1 or neighbor[1] > 9:
                    continue
                elif neighbor in obstacle_points:
                    continue
                else:
                    neighbor_weigh[str(neighbor[0])+str(neighbor[1])] = 1
            graph[str(i)+str(j)] = neighbor_weigh
    result = djkstra(graph, str(start_point[0])+str(start_point[1]), str(end_point[0])+str(end_point[1]))
    tmp = result[2].split('-')
    path_vertices = []
    for elem in tmp:
        path_vertices.append([int(elem[0]), int(elem[1])])
    print("path_finding_d result:" ,path_vertices)
    return path_vertices

In [31]:
def walk_to_target(path_vertices, face_direction):
    if len(path_vertices) == 1:
        return
    start_point = path_vertices[0]

    # # face_direction: 0表示向上，1表示向右，2表示向下，3表示向左
    # if start_point[0] == 1:
    #     face_direction = 2
    # elif start_point[0] == 9:
    #     face_direction = 0
    # elif start_point[1] == 1:
    #     face_direction = 1
    # elif start_point[1] == 9:
    #     face_direction = 3
    # else:
    #     raise SystemExit('Wrong Start Point!')

    for i in range(len(path_vertices) - 1):
        current_point = path_vertices[i]
        next_point = path_vertices[i + 1]
        step = [
            next_point[0] - current_point[0], next_point[1] - current_point[1]
        ]
        if step[0] == 1:
            need_direction = 2
        elif step[0] == -1:
            need_direction = 0
        elif step[1] == 1:
            need_direction = 1
        elif step[1] == -1:
            need_direction = 3
        else:
            raise SystemExit('Wrong Direction!')
        angle = need_direction - face_direction
        if angle > 2:
            angle = angle - 4
        if angle < -2:
            angle = angle + 4
        # 旋转
        turn(angle)
        face_direction = need_direction
        # 前进
        current_location = walk_step(current_point, next_point)
        
    return face_direction, current_location

In [32]:
def center2(): # I want to get close enough
    ff = 0
    lrmin= 250 #######TODO
    rrmin=250  #######TODO
    mk_min = 10
    rmax = 100   #######TODO
    mrmax = 150
    mrmin = 0
    while(ff==0):
        takepic()
        img = cv.imread('/home/xilinx/jupyter_notebooks/common/data/webcam.jpg')
        [lk,lr,mk,mr,rk,rr] = houfline(img)
        if(mk==0):
            print('finish')
            break
            #run_action('Back2Run')
            #wait_req()
            #print('back because mk==0')
            #run_action('Stand')
            #wait_req()
        elif(abs(mk)<mk_min):
            if(mk>0):
                #left
                run_action('turn001L')
                wait_req()
            elif(mk<0):    
                run_action('turn001R')
                wait_req()
        elif(mr<mrmin):
            #run_action('Back2Run')
            #run_action('fastForward03')
            #wait_req()
            print('close enough')
            return 0
            #run_action('Stand')
            #wait_req()
        elif(mr>mrmax):
            run_action('Forwalk02')########TODO
            #run_action('fastForward03')
            wait_req()
        elif(lk!=0 and rk!=0):
            if(abs(lr-rr)>rmax):
                if(lr>rr):
                    run_action('Left3move')
                    wait_req()
                elif(lr<rr):
                    run_action('Right3move')
                    wait_req()
            else:
                ff = 1        
        elif(lk==0 or rk == 0):
            if(lk!=0):
                if(lr<lrmin):
                    run_action('Right3move')
                    wait_req()    
            elif(rr!=0):
                if(rr<rrmin):
                     run_action('Left3move')
                     wait_req()

def center1(): # this version only judge left or right
    ff = 0
    lrmin= 250 #######TODO
    rrmin=250  #######TODO
    mk_min = 10
    rmax = 100   #######TODO
    mrmax = 250
    mrmin = 150
    while(ff==0):
        takepic()
        img = cv.imread('/home/xilinx/jupyter_notebooks/common/data/webcam.jpg')
        [lk,lr,mk,mr,rk,rr] = houfline(img)
        if(mk==0):
            run_action('Back2Run')
            wait_req()
            print('back because mk==0')
            run_action('Stand')
            wait_req()
        elif(abs(mk)<mk_min):
            if(mk>0):
                #left
                run_action('turn001L')
                wait_req()
            elif(mk<0):    
                run_action('turn001R')
                wait_req()
        elif(mr<mrmin):
            run_action('Back2Run')
            #run_action('fastForward03')
            wait_req()
            print('back because too close')
            run_action('Stand')
            wait_req()
        elif(mr>mrmax):
            run_action('Forwalk02')########TODO
            #run_action('fastForward03')
            wait_req()
        elif(lk!=0 and rk!=0):
            if(abs(lr-rr)>rmax):
                if(lr>rr):
                    run_action('Left3move')
                    wait_req()
                elif(lr<rr):
                    run_action('Right3move')
                    wait_req()
            else:
                ff = 1        
        elif(lk==0 or rk == 0):
            if(lk!=0):
                if(lr<lrmin):
                    run_action('Right3move')
                    wait_req()    
            elif(rr!=0):
                if(rr<rrmin):
                     run_action('Left3move')
                     wait_req()        
    
    
    
def finish():
    center2()
    run_action('Wanyao20')
    wait_req()
    run_action('Stand')
    wait_req()
    run_action('Back2Run')
    wait_req()
    run_action('Stand')
    wait_req()
    center()

In [33]:
import copy
def des_position(number):
    des_pos = []
    destination_dict = {
        1: [[2, 2]],
        2: [[2, 5]],
        3: [[2, 8]],
        4: [[5, 2]],
        5: [[5, 5]],
        6: [[5, 8]],
        7: [[8, 2]],
        8: [[8, 5]],
        9: [[8, 8]]
    }
    for n in number:
        des_pos = des_pos + destination_dict[n]
    print(des_pos)
    return des_pos

####### TODO
def des_to_target(des,face):
    # 0-下，1-右，2-上，3-左
    out = copy.deepcopy(des)
    if(face == 0):
        out[0] = out[0] + 1
    elif(face == 1):
        out[1] = out[1] + 1
    elif(face == 2):
        out[0] = out[0] - 1    
    elif(face == 3):
        out[1] = out[1] - 1
 
    dtt_dict = {
        '[2, 2]': [2, 1],
        '[2, 5]': [2, 6],
        '[2, 8]': [2, 9],
        '[5, 2]': [5, 1],
        '[5, 5]': [5, 6],
        '[5, 8]': [4, 8],
        '[8, 2]': [8, 1],
        '[8, 5]': [8, 6],
        '[8, 8]': [8, 9]
    }
    #return dtt_dict[str(destination)]
    print("target is ",out)
    return out

In [36]:
    #print(target_animal)
    # test data
    start_point = [3, 9]
    targets = des_position([1,7])
    targets1 = des_position([1,7])
    end_point = [7, 1]
    barrier = [2, 8]
    direc = [2, 3]
    t = direc[0]
    r = direc[1]
    # real data 
    # # cmd = 'robot1'
    
    #Location, barrier, Target, direc,start_point, end_point = get_data_frame(cmd)
    # targets = des_position(Target)
    
    # 挑选合适的起始点位
    print('before dirc',targets)
    target0_point = des_to_target(targets[0],direc[0])
    target1_point = des_to_target(targets[1],direc[1])
    print('before and after',targets,target0_point)
    path_1 = path_finding_d(start_point, target0_point, barrier)
    print("Path 1:",path_1)
    path_2 = path_finding_d(start_point, target1_point, barrier)
    print("Path 2:",path_2)
    path_3 = path_finding_d(target0_point, end_point, barrier)
    print("Path 3:",path_3)
    path_4 = path_finding_d(target1_point, end_point, barrier)
    print("Path 4:",path_4)
    print('after path',targets,target0_point)
    if (len(path_1) + len(path_3)) > 12 and (len(path_1) + len(path_3)) < 19 and (len(path_2) + len(path_4)) > 12 and (len(path_2) + len(path_4)) < 19:
        if len(path_1) > len(path_2):
            targets = [targets[1], targets[0]]
            targets1 = [targets1[1],targets1[0]]
            direc[0]=r
            direc[1]=t
    else:
        if (len(path_2) + len(path_4)) > 12 and (len(path_2) + len(path_4)) < 19:
            targets = [targets[1], targets[0]]
            targets1 = [targets1[1],targets1[0]]
            direc[0]=r
            direc[1]=t
    print('direc,target are after change',direc,targets)        
    # 计算起始朝向
    # face_direction: 0表示向上，1表示向右，2表示向下，3表示向左
    if start_point[0] == 1:
        face_direction = 2
    elif start_point[0] == 9:
        face_direction = 0
    elif start_point[1] == 1:
        face_direction = 1
    elif start_point[1] == 9:
        face_direction = 3
    else:
        raise SystemExit('Wrong Start Point!')
    # 确定台阶侧面以开始搜寻朝向并抵达正面
    detect_point = des_to_target(targets[0],direc[0])
    #detect_point = target0_point
    path_vertices = path_finding_d(start_point, detect_point, barrier)
    current_direction, current_location = walk_to_target(path_vertices, face_direction)

    #current_direction, target_block = stair_detect.stair_detect(current_direction, targets[0], current_location, barrier)
    #path_vertices = Path.path_finding_d(current_location, target_block, barrier)
    #current_direction, current_location = Path.walk_to_target(path_vertices, current_direction)

    # 朝向
    step = [
            targets1[0][0] - current_location[0], targets1[0][1] - current_location[1]
        ]
    print('step are ',step)
    if step[0] == 1:
        need_direction = 2
    elif step[0] == -1:
        need_direction = 0
    elif step[1] == 1:
        need_direction = 1
    elif step[1] == -1:
        need_direction = 3
    else:
        raise SystemExit('Wrong Direction!')
    angle = need_direction - current_direction
    if angle > 2:
        angle = angle - 4
    if angle < -2:
        angle = angle + 4
    turn(angle)
    current_direction = need_direction

    center()

    # TODO: 走的步数不知道
    ####### utils.walk()
    # utils.fall()h
    # 识别小动物
    #pred = find_duck()
    finish()

    # test
    if(True): ##### DUCK !!!
        detect_point = des_to_target(targets[1],direc[1])
        #detect_point = targets[1]
        path_vertices = path_finding_d(current_location, detect_point, barrier)
        current_direction, current_location = walk_to_target(path_vertices, face_direction)

        #current_direction, target_block = stair_detect.stair_detect(current_direction, targets[1], current_location, barrier)
        #path_vertices = Path.path_finding_d(current_location, target_block, barrier)
        #current_direction, current_location = Path.walk_to_target(path_vertices, current_direction)
        # 朝向
        step = [
                    targets[1][0] - current_location[0], targets[1][1] - current_location[1]
                ]
        print('second step are ',step)
        if step[0] == 1:
            need_direction = 2
        elif step[0] == -1:
            need_direction = 0
        elif step[1] == 1:
            need_direction = 1
        elif step[1] == -1:
            need_direction = 3
        else:
            raise SystemExit('Wrong Direction!')
        angle = need_direction - current_direction
        if angle > 2:
            angle = angle - 4
        if angle < -2:
            angle = angle + 4
        turn(angle)
        current_direction = need_direction

        finish()

        # 上台阶


        # 上台阶
        path_vertices = path_finding_d(current_location, end_point, barrier)
        current_direction, current_location = walk_to_target(path_vertices, current_direction)


[[2, 2], [8, 2]]
[[2, 2], [8, 2]]
before dirc [[2, 2], [8, 2]]
target is  [1, 2]
target is  [8, 1]
before and after [[2, 2], [8, 2]] [1, 2]
path_finding_d result: [[3, 9], [3, 8], [3, 7], [3, 6], [3, 5], [3, 4], [2, 4], [2, 3], [1, 3], [1, 2]]
Path 1: [[3, 9], [3, 8], [3, 7], [3, 6], [3, 5], [3, 4], [2, 4], [2, 3], [1, 3], [1, 2]]
path_finding_d result: [[3, 9], [4, 9], [4, 8], [4, 7], [5, 7], [5, 6], [6, 6], [6, 5], [6, 4], [6, 3], [6, 2], [6, 1], [7, 1], [8, 1]]
Path 2: [[3, 9], [4, 9], [4, 8], [4, 7], [5, 7], [5, 6], [6, 6], [6, 5], [6, 4], [6, 3], [6, 2], [6, 1], [7, 1], [8, 1]]
path_finding_d result: [[1, 2], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1]]
Path 3: [[1, 2], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1]]
path_finding_d result: [[8, 1], [7, 1]]
Path 4: [[8, 1], [7, 1]]
after path [[2, 2], [8, 2]] [1, 2]
direc,target are after change [2, 3] [[2, 2], [8, 2]]
target is  [1, 2]
path_finding_d result: [[3, 9], [3, 8], [3, 7], [3, 6], [3, 5], [3, 4], [2, 4

read REQ finished!
back because too close
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b' '
test_read b'\x13'
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebo

read REQ finished!
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[0.48484848484848486, 183.94381718622918], [-8.3793103448275854, 159.06468821699201]]
2
all [0.48484848484848486, 183.94381718622918, -8.3793103448275854, 159.06468821699201, 0, 0]
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was sp

/usr/lib/python3/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in int_scalars


Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[52.625, 224.04506409372615], [-62.666666666666664, 215.76508371790547], [0.46276595744680848, 208.39032411056169]]
3
all [0.46276595744680848, 208.39032411056169, -62.666666666666664, 215.76508371790547, 0, 0]
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[53.625, 224.11954291715608], [0.46341463414634149, 208.10607180106524], [-0.6717557251908397, 257.50067150229279]]
3
all [0.46341463414634149, 208.10607180106524, 53.625, 224.11954291715608, -0.6717557251908397, 257.500671

...

read REQ finished!
back because too close
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b' '
test_read b'\x13'
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebo

Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[-0.4434389140271493, 146.96343779639511], [11.428571428571429, 203.87104428557953], [-1.6585365853658536, 352.36246770042544]]
3
all [0, 0, 11.428571428571429, 203.87104428557953, -0.4434389140271493, 146.96343779639511]
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_noteboo

/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[0.40476190476190477, 189.07507973530471]]
1
all [0.40476190476190477, 189.07507973530471, 0, 0, 0, 0]
finish
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'


--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[-5.7254901960784315, 81.441787928754763]]
1
all [0, 0, -5.7254901960784315, 81.441787928754763, 0, 0]
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[14.346153846153847, 66.249436

KeyboardInterrupt: 

In [56]:
def center3():
    wait_req()

test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b' '
test_read b'\x13'
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
T

/usr/lib/python3/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in int_scalars


test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!


KeyboardInterrupt: 

In [29]:
run_action('fastForward03')
wait_req()

test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!


In [19]:
    # test data
    start_point = [7, 1]
    targets = des_position([2, 8])
    targets1 = des_position([2,8])
    end_point = [3, 9]
    barrier = [4, 6]
    direc = [0, 2]
    t = direc[0]
    r = direc[1]
    # real data 
    # # cmd = 'robot1'
    
    #Location, barrier, Target, direc,start_point, end_point = get_data_frame(cmd)
    # targets = des_position(Target)
    
    # 挑选合适的起始点位
    print('before dirc',targets)
    target0_point = des_to_target(targets[0],direc[0])
    target1_point = des_to_target(targets[1],direc[1])
    print('before and after',targets,target0_point)
    path_1 = path_finding_d(start_point, target0_point, barrier)
    print("Path 1:",path_1)
    path_2 = path_finding_d(start_point, target1_point, barrier)
    print("Path 2:",path_2)
    path_3 = path_finding_d(target0_point, end_point, barrier)
    print("Path 3:",path_3)
    path_4 = path_finding_d(target1_point, end_point, barrier)
    print("Path 4:",path_4)
    path_5 = path_finding_d(target0_point, target1_point, barrier)
    ccc = len(path_5)
    print('after path',targets,target0_point)
    if((len(path_1)+len(path_4)) >= (len(path_2)+len(path_3))  ):
            targets = [targets[1], targets[0]]
            targets1 = [targets1[1],targets1[0]]
            direc[0]=r
            direc[1]=t       
    

    print('direc,target are after change',direc,targets)        
    # 计算起始朝向
    # face_direction: 0表示向上，1表示向右，2表示向下，3表示向左
    if start_point[0] == 1:
        face_direction = 2
    elif start_point[0] == 9:
        face_direction = 0
    elif start_point[1] == 1:
        face_direction = 1
    elif start_point[1] == 9:
        face_direction = 3
    else:
        raise SystemExit('Wrong Start Point!')
    # 确定台阶侧面以开始搜寻朝向并抵达正面
    detect_point = des_to_target(targets[0],direc[0])
    #detect_point = target0_point
    path_vertices = path_finding_d(start_point, detect_point, barrier)
    current_direction, current_location = walk_to_target(path_vertices, face_direction)

    #current_direction, target_block = stair_detect.stair_detect(current_direction, targets[0], current_location, barrier)
    #path_vertices = Path.path_finding_d(current_location, target_block, barrier)
    #current_direction, current_location = Path.walk_to_target(path_vertices, current_direction)

    # 朝向
    step = [
            targets1[0][0] - current_location[0], targets1[0][1] - current_location[1]
        ]
    print('step are ',step)
    if step[0] == 1:
        need_direction = 2
    elif step[0] == -1:
        need_direction = 0
    elif step[1] == 1:
        need_direction = 1
    elif step[1] == -1:
        need_direction = 3
    else:
        raise SystemExit('Wrong Direction!')
    angle = need_direction - current_direction
    if angle > 2:
        angle = angle - 4
    if angle < -2:
        angle = angle + 4
    turn(angle)
    current_direction = need_direction

    center()

    # TODO: 走的步数不知道
    ####### utils.walk()
    # utils.fall()h
    # 识别小动物
    #pred = find_duck()
    finish()

    # test
    if(True): ##### DUCK !!!
        detect_point = des_to_target(targets[1],direc[1])
        #detect_point = targets[1]
        path_vertices = path_finding_d(current_location, detect_point, barrier)
        current_direction, current_location = walk_to_target(path_vertices, face_direction)

        #current_direction, target_block = stair_detect.stair_detect(current_direction, targets[1], current_location, barrier)
        #path_vertices = Path.path_finding_d(current_location, target_block, barrier)
        #current_direction, current_location = Path.walk_to_target(path_vertices, current_direction)
        # 朝向
        step = [
                    targets[1][0] - current_location[0], targets[1][1] - current_location[1]
                ]
        print('second step are ',step)
        if step[0] == 1:
            need_direction = 2
        elif step[0] == -1:
            need_direction = 0
        elif step[1] == 1:
            need_direction = 1
        elif step[1] == -1:
            need_direction = 3
        else:
            raise SystemExit('Wrong Direction!')
        angle = need_direction - current_direction
        if angle > 2:
            angle = angle - 4
        if angle < -2:
            angle = angle + 4
        turn(angle)
        current_direction = need_direction

        finish()

        # 上台阶


        # 上台阶
        path_vertices = path_finding_d(current_location, end_point, barrier)
        current_direction, current_location = walk_to_target(path_vertices, current_direction)


[[2, 5], [8, 5]]
[[2, 5], [8, 5]]
before dirc [[2, 5], [8, 5]]
target is  [3, 5]
target is  [7, 5]
before and after [[2, 5], [8, 5]] [3, 5]
path_finding_d result: [[7, 1], [7, 2], [7, 3], [6, 3], [6, 4], [5, 4], [4, 4], [3, 4], [3, 5]]
Path 1: [[7, 1], [7, 2], [7, 3], [6, 3], [6, 4], [5, 4], [4, 4], [3, 4], [3, 5]]
path_finding_d result: [[7, 1], [7, 2], [7, 3], [7, 4], [7, 5]]
Path 2: [[7, 1], [7, 2], [7, 3], [7, 4], [7, 5]]
path_finding_d result: [[3, 5], [3, 6], [3, 7], [3, 8], [3, 9]]
Path 3: [[3, 5], [3, 6], [3, 7], [3, 8], [3, 9]]
path_finding_d result: [[7, 5], [7, 6], [7, 7], [7, 8], [7, 9], [6, 9], [5, 9], [4, 9], [3, 9]]
Path 4: [[7, 5], [7, 6], [7, 7], [7, 8], [7, 9], [6, 9], [5, 9], [4, 9], [3, 9]]
path_finding_d result: [[3, 5], [3, 4], [4, 4], [5, 4], [6, 4], [7, 4], [7, 5]]
after path [[2, 5], [8, 5]] [3, 5]
direc,target are after change [2, 0] [[8, 5], [2, 5]]
target is  [7, 5]
path_finding_d result: [[7, 1], [7, 2], [7, 3], [7, 4], [7, 5]]
test_read b'\xff'
test_read b

ValueError: not enough values to unpack (expected 6, got 3)

In [28]:
turn(-1)

test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b' '
test_read b'\x13'
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
--- Opening /dev/video2...
T

/usr/lib/python3/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in int_scalars


test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b' '
test_read b'\x13'
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0c'
test_read b'\x04'
test_read b'R'
test_read b'E'
test_read b'Q'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b'\x8e'
test_read b'\xa3'
read REQ finished!
send action ok!
read REQ finished!
back because mk==0
test_read b'\xff'
test_read b'\xff'
test_read b'\x00'
test_read b'\x0e'
test_read b'\x06'
test_read b'E'
test_read b'R'
test_read b'R'
test_read b'O'
test_read b'R'
test_read b'\x00'
test_read b'\x01'
test_read b'\x03'
test_read b'\x02'
test_read b'\x00'
test_read b'\x00'
test_read b' '
test_read b'\x13'
test_read b'\xff'
test_read b'\xff'


ValueError: not enough values to unpack (expected 6, got 3)